In [1]:
import numpy as np
import faiss
import time
import pandas as pd
from os import path, makedirs
import torch
import sys
sys.path.append('../')
from utils import load_embeddings

## Configuration Variables

In [2]:
root = '../../../inference_array/resnet50/' # path to database and queryset
D = 2048 # embedding dim
hnsw_max_neighbors = 32 # M for HNSW, default=32
pq_num_subvectors = 32 # m for HNSW+PQ

model = 'mrl' # mrl, rr
dataset = '1K' # 1K, 4K, V2
index_type = 'hnsw32' # exactl2, hnsw32, #'hnswpq_M'+str(hnsw_max_neighbors)+'_pq-m'+str(pq_num_subvectors)

k = 2048 # shortlist length, default is set to the max supported by FAISS
nesting_list = [8, 16, 32, 64] # embedding dim to loop over

## FAISS Index Building and NN Search

In [3]:
if index_type == 'exactl2' and torch.cuda.device_count() > 0:
    use_gpu = True # GPU inference for exact search
else:
    use_gpu = False # GPU inference for HNSW is currently not supported by FAISS

In [4]:
def get_nn(index_type, nesting_list, m=8):
    for retrieval_dim in nesting_list:
        if retrieval_dim > D:
            continue
        
        if index_type == 'hnswpq_M'+str(hnsw_max_neighbors)+'_pq-m'+str(m) and retrieval_dim < m:
            continue
            
        if not path.isdir(root+'index_files/'+model+'/'):
            makedirs(root+'index_files/'+model+'/')
        index_file = root+'index_files/'+model+'/'+dataset+'_'+str(retrieval_dim)+'dim_'+index_type+'.index'

        _, _, _, _, xb, xq = load_embeddings(model, dataset, retrieval_dim)

        # Load or build index
        if path.exists(index_file): # Load index
            print("Loading index file: " + index_file.split("/")[-1])
            cpu_index = faiss.read_index(index_file)
            
        else: # Build index
            print("Generating index file: " + index_file)

            d = xb.shape[1] # dimension

            start = time.time()
            if index_type == 'exactl2':
                print("Building Exact L2 Index")
                cpu_index = faiss.IndexFlatL2(d) # build the index
            elif index_type == 'hnswpq_M'+str(hnsw_max_neighbors)+'_pq-m'+str(m):
                print("Building D%d + HNSW%d + PQ%d Index" % (d, hnsw_max_neighbors, m))
                cpu_index = faiss.IndexHNSWPQ(d, m, hnsw_max_neighbors)
                cpu_index.train(xb)
            elif index_type == f'hnsw{hnsw_max_neighbors}':
                print("Building HNSW%d Index" % hnsw_max_neighbors)
                cpu_index = faiss.IndexHNSWFlat(d, hnsw_max_neighbors)
            else:
                raise Exception(f"Unsupported Index: {index_type}")
                
            cpu_index.add(xb) # add vectors to the index
            faiss.write_index(cpu_index, index_file)
            print("GPU Index build time= %0.3f sec" % (time.time() - start))

        if use_gpu:
            index = faiss.index_cpu_to_all_gpus(cpu_index)
        else:
            index = cpu_index
        
        # Iterate over efSearch (HNSW search probes)
        efsearchlist = [16]
        for efsearch in efsearchlist:
            start = time.time()
            if index_type in ['hnsw32', 'hnswpq_M'+str(hnsw_max_neighbors)+'_pq-m'+str(m)]:
                index.hnsw.efSearch = efsearch
                print("Searching with Efsearch =", index.hnsw.efSearch)
            Dist, Ind = index.search(xq, k)
            # print("GPU %d-NN search time= %f sec" % (k, time.time() - start))
            if not path.isdir(root+"neighbors/"+model+'/'+index_type):
                makedirs(root+"neighbors/"+model+'/'+index_type)
            nn_dir = root+"neighbors/"+model+'/'+index_type+"/"+index_type+'_efsearch'+str(efsearch)+"_"+str(k)+"shortlist_"+dataset+"_d"+str(retrieval_dim)+".csv"
            pd.DataFrame(Ind).to_csv(nn_dir, header=None, index=None)
            
        del index, Dist, Ind

In [6]:
nesting_list = [8, 16, 32, 64]
get_nn(index_type, nesting_list)

Generating index file: ../../../inference_array/resnet50/index_files/mrl1K_8dim_hnsw32.index
Building HNSW32 Index
GPU Index build time= 57.573 sec
Searching with Efsearch = 16
13010622
Generating index file: ../../../inference_array/resnet50/index_files/mrl1K_16dim_hnsw32.index
Building HNSW32 Index
GPU Index build time= 71.297 sec
Searching with Efsearch = 16
15721346
Generating index file: ../../../inference_array/resnet50/index_files/mrl1K_32dim_hnsw32.index
Building HNSW32 Index
GPU Index build time= 74.260 sec
Searching with Efsearch = 16
16834028
Generating index file: ../../../inference_array/resnet50/index_files/mrl1K_64dim_hnsw32.index
Building HNSW32 Index
GPU Index build time= 77.627 sec
Searching with Efsearch = 16
17557581


In [5]:
# k = 40 to generate Exact Ground Truth for 40-Recall@2048
# nesting_list = [D] # fixed embedding dimension for RR models
pq_m_values = [8, 16] # loop over PQ m values

for m in pq_m_values:
    index_type = 'hnswpq_M'+str(hnsw_max_neighbors)+'_pq-m'+str(m)
    get_nn(index_type, nesting_list, m)

Loading index file: 1K_8dim_hnswpq_M32_pq-m8.index
Searching with Efsearch = 16
Loading index file: 1K_16dim_hnswpq_M32_pq-m8.index
Searching with Efsearch = 16
Loading index file: 1K_32dim_hnswpq_M32_pq-m8.index
Searching with Efsearch = 16
Loading index file: 1K_64dim_hnswpq_M32_pq-m8.index
Searching with Efsearch = 16
Generating index file: ../../../inference_array/resnet50/index_files/mrl/1K_16dim_hnswpq_M32_pq-m16.index
Building D16 + HNSW32 + PQ16 Index
GPU Index build time= 180.430 sec
Searching with Efsearch = 16
Generating index file: ../../../inference_array/resnet50/index_files/mrl/1K_32dim_hnswpq_M32_pq-m16.index
Building D32 + HNSW32 + PQ16 Index
GPU Index build time= 180.994 sec
Searching with Efsearch = 16
Generating index file: ../../../inference_array/resnet50/index_files/mrl/1K_64dim_hnswpq_M32_pq-m16.index
Building D64 + HNSW32 + PQ16 Index
GPU Index build time= 182.374 sec
Searching with Efsearch = 16
